In [1]:
import json
import json
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

In [2]:
with open('../processed/chroma_documents.json', 'r') as f:
    chroma_documents = json.load(f)

In [3]:
grande_ecole = list()
specialized_schools = list()
for doc in chroma_documents:
    if doc['metadata']['school_type'] == 'Grande Ecole':
        grande_ecole.append(doc)
    elif doc['metadata']['school_type'] == 'Ecole Spécialisée':
        specialized_schools.append(doc)

In [4]:
print(len(grande_ecole), len(specialized_schools))

591 730


In [5]:
#save each
with open('../processed/grande_ecole.json', 'w') as f:
    json.dump(grande_ecole, f, indent=2, ensure_ascii=False)
with open('../processed/specialized_schools.json', 'w') as f:
    json.dump(specialized_schools, f, indent=2, ensure_ascii=False)



In [6]:
model_name = "intfloat/multilingual-e5-large-instruct"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/opt/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [10]:
vdbs = ['ecole_specialisee_db', 'grande_ecole_db']

chroma_docs  = grande_ecole

vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="./grande_ecole_db",  # Where to save data locally, remove if not necessary
)

documents = [Document( page_content=doc['page_content'],
                      metadata=doc['metadata']) for doc in chroma_docs]

vector_store.add_documents(documents=documents)


InternalError: Query error: Database error: error returned from database: (code: 1032) attempt to write a readonly database